In [7]:
import numpy as np
import pandas as pd
import requests
import math
from scipy import stats
import xlsxwriter
from secrets import IEX_CLOUD_API_TOKEN

In [8]:
stocks = pd.read_csv('sp_500_stocks.csv')
stocks

,Ticker
0,A
1,AAL
2,AAP
3,AAPL
4,ABBV
...,...
500,YUM
501,ZBH
502,ZBRA
503,ZION


In [9]:
symbol = "AAPL"
api_url = f"https://sandbox.iexapis.com/stable/stock/{symbol}/stats/?token={IEX_CLOUD_API_TOKEN}"    
data = requests.get(api_url).json()
data

{'companyName': 'Apple Inc',
 'marketcap': 2401857607474,
 'week52high': 160.25,
 'week52low': 110.65,
 'week52highSplitAdjustOnly': 162.87,
 'week52lowSplitAdjustOnly': 111.75,
 'week52change': 0.232215855217735,
 'sharesOutstanding': 17003374400,
 'float': 0,
 'avg10Volume': 68000664,
 'avg30Volume': 82779879,
 'day200MovingAvg': 141.09,
 'day50MovingAvg': 148.8,
 'employees': 150135,
 'ttmEPS': 5.2,
 'ttmDividendRate': 0.857150901492781,
 'dividendYield': 0.006039463337230438,
 'nextDividendDate': '',
 'exDividendDate': '2021-08-01',
 'nextEarningsDate': '2021-10-25',
 'peRatio': 28.274933406131154,
 'beta': 1.4009524535723772,
 'maxChangePercent': 55.25699588735628,
 'year5ChangePercent': 4.403137323000014,
 'year2ChangePercent': 1.5391888461600067,
 'year1ChangePercent': 0.20796301870761,
 'ytdChangePercent': 0.09829234675498427,
 'month6ChangePercent': 0.081266599026744,
 'month3ChangePercent': -0.02308372523311059,
 'month1ChangePercent': -0.028783614701572282,
 'day30ChangePerc

In [21]:
def chunks(lst, n):
    """Yield successive n-sized chunks from lst."""
    for i in range(0, len(lst), n):
        yield lst[i:i + n]
symbol_groups = list(chunks(stocks['Ticker'], 100))
#print(symbol_groups)
symbol_strings = list()
for i in range(0, len(symbol_groups)):
    symbol_strings.append(','.join(symbol_groups[i]))
    print(symbol_strings[i])
    
my_columns = ["Ticker", "Price", "One-Year Price Return", "Number of Shares to Buy"]

A,AAL,AAP,AAPL,ABBV,ABC,ABMD,ABT,ACN,ADBE,ADI,ADM,ADP,ADSK,AEE,AEP,AES,AFL,AIG,AIV,AIZ,AJG,AKAM,ALB,ALGN,ALK,ALL,ALLE,ALXN,AMAT,AMCR,AMD,AME,AMGN,AMP,AMT,AMZN,ANET,ANSS,ANTM,AON,AOS,APA,APD,APH,APTV,ARE,ATO,ATVI,AVB,AVGO,AVY,AWK,AXP,AZO,BA,BAC,BAX,BBY,BDX,BEN,BF.B,BIIB,BIO,BK,BKNG,BKR,BLK,BLL,BMY,BR,BRK.B,BSX,BWA,BXP,C,CAG,CAH,CARR,CAT,CB,CBOE,CBRE,CCI,CCL,CDNS,CDW,CE,CERN,CF,CFG,CHD,CHRW,CHTR,CI,CINF,CL,CLX,CMA,CMCSA
CME,CMG,CMI,CMS,CNC,CNP,COF,COG,COO,COP,COST,COTY,CPB,CPRT,CRM,CSCO,CSX,CTAS,CTL,CTSH,CTVA,CTXS,CVS,CVX,CXO,D,DAL,DD,DE,DFS,DG,DGX,DHI,DHR,DIS,DISCA,DISCK,DISH,DLR,DLTR,DOV,DOW,DPZ,DRE,DRI,DTE,DUK,DVA,DVN,DXC,DXCM,EA,EBAY,ECL,ED,EFX,EIX,EL,EMN,EMR,EOG,EQIX,EQR,ES,ESS,ETFC,ETN,ETR,EVRG,EW,EXC,EXPD,EXPE,EXR,F,FANG,FAST,FB,FBHS,FCX,FDX,FE,FFIV,FIS,FISV,FITB,FLIR,FLS,FLT,FMC,FOX,FOXA,FRC,FRT,FTI,FTNT,FTV,GD,GE,GILD
GIS,GL,GLW,GM,GOOG,GOOGL,GPC,GPN,GPS,GRMN,GS,GWW,HAL,HAS,HBAN,HBI,HCA,HD,HES,HFC,HIG,HII,HLT,HOLX,HON,HPE,HPQ,HRB,HRL,HSIC,HST,HSY,HUM,HWM,IBM,ICE,IDXX,IEX,IFF,ILM

In [35]:
final_dataframe = pd.DataFrame(columns = my_columns)

for symbol_string in symbol_strings:
    batch_api_call_url = f'https://sandbox.iexapis.com/stable/stock/market/batch/?types=stats,quote&symbols={symbol_string}&token={IEX_CLOUD_API_TOKEN}'
    data = requests.get(batch_api_call_url).json()
    for symbol in symbol_string.split(','):
        final_dataframe = final_dataframe.append(
            pd.Series(
                [
                symbol,
                data[symbol]['quote']['latestPrice'],
                data[symbol]['stats']['year1ChangePercent'],
                'N/A'],
            index = my_columns),
        ignore_index=True)
final_dataframe


,Ticker,Price,One-Year Price Return,Number of Shares to Buy
0,A,156.51,0.481672,N/A
1,AAL,20.91,0.66443,N/A
2,AAP,218.55,0.385505,N/A
3,AAPL,146.50,0.209309,N/A
4,ABBV,109.58,0.350424,N/A
...,...,...,...,...
500,YUM,128.02,0.293849,N/A
501,ZBH,153.35,-0.001844,N/A
502,ZBRA,531.60,0.82321,N/A
503,ZION,65.12,1.071634,N/A


In [40]:
final_dataframe.sort_values("One-Year Price Return", ascending=False, inplace=True)
final_dataframe = final_dataframe[:50]
final_dataframe.reset_index(inplace=True)
final_dataframe

,index,Ticker,Price,One-Year Price Return,Number of Shares to Buy
0,148,DVN,41.50,3.405927,N/A
1,317,MRO,16.65,2.908738,N/A
2,175,FANG,112.43,2.643854,N/A
3,275,LB,80.00,2.285599,N/A
4,357,OXY,32.20,2.078738,N/A
5,299,MCHP,73.51,1.660134,N/A
6,410,SIVB,694.52,1.56151,N/A
7,23,ALB,239.69,1.472687,N/A
8,251,IT,315.84,1.470315,N/A
9,195,FTNT,319.60,1.457204,N/A


In [45]:
def portfolio_input():
    global portfolio_size
    portfolio_size = input("Enter the size of your portfolio: ")
    
    try:
        float(portfolio_size)
    except ValueError:
        print("THat is not a number! \nPlease try again: ")
        portfolio_size = input("Enter the size of your portfolio: ")

portfolio_input()

Enter the size of your portfolio: 1000000


In [47]:
position_size = float(portfolio_size)/len(final_dataframe.index)
print(position_size)

20000.0


In [48]:
for i in range(0, len(final_dataframe)):
    final_dataframe.loc[i, "Number of Shares to Buy"] = math.floor(position_size/final_dataframe.loc[i, "Price"])

final_dataframe

Z:\Projects\strategy_index_screener\.venv\lib\site-packages\pandas\core\indexing.py:1817: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self._setitem_single_column(loc, value, pi)


,index,Ticker,Price,One-Year Price Return,Number of Shares to Buy
0,148,DVN,41.50,3.405927,481
1,317,MRO,16.65,2.908738,1201
2,175,FANG,112.43,2.643854,177
3,275,LB,80.00,2.285599,250
4,357,OXY,32.20,2.078738,621
5,299,MCHP,73.51,1.660134,272
6,410,SIVB,694.52,1.56151,28
7,23,ALB,239.69,1.472687,83
8,251,IT,315.84,1.470315,63
9,195,FTNT,319.60,1.457204,62


In [50]:
print("""
Building HQM - high quality momentum - Strategy
1-month price returns
3-month price returns
6-month price returns
1-year price returns
""")


Building HQM - high quality momentum - Strategy
1-month price returns
3-month price returns
6-month price returns
1-year price returns

